# OPTUNA _ XLM

In [17]:
import pandas as pd
train_df = train_processed
dev_df = dev_processed

In [18]:
#set device
import torch
# device = torch.device("cpu")
device  =  torch. device('cuda:0'  if  torch. cuda. is_available() else  'cpu')
print(f"device is {device}")

device is cuda:0


In [54]:
!pip install optuna
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install pysentimiento
# !pip install --upgrade pip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
import optuna
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import os.path
from os import path

# Create function for printing
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)

2024-05-06 14:45:25.599140: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 14:45:25.599278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 14:45:25.734015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
import pandas as pd
import numpy as np
import random
import  matplotlib. pyplot  as  plt
from tqdm import tqdm

import torch
import torch.optim as optim
import  torch. nn. functional  as  F

from  IPython. display  import  clear_output
import warnings
warnings. filterwarnings('ignore')

#setting device agnostic version
# device = "cpu"
device  =  torch. device('cuda:0'  if  torch. cuda. is_available() else  'cpu')
print(f"computation will run on {device} now")

computation will run on cuda:0 now


In [9]:
#relevant functions
import re
# Create function for printing
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)

def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def tweet_preprocessor_spanish(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="es", shorten=3)

def tweet_preprocessor_english(text):
    """
    imported from pysentimiento
    does the following:
    1. changes to @usario
    2. shortens characters repetitions to 3 (can be changed to two or more)
    3. converts emojis to text type of need
    4. normalises laughter expressions
    5. handles hashtags -- removes the #
    """
    from pysentimiento.preprocessing import preprocess_tweet
    return preprocess_tweet(text, lang="en", shorten=3)

#instantiate label encoders
from sklearn.preprocessing import LabelEncoder
task_encoder = LabelEncoder()

def task_hard_encode(df):
    task_encoder.fit(all_task_hard_labels)
    df['task2_hard_labels'] = task_encoder.transform(df['task2_hard_labels'])
    return df

def task_hard_decode(df):
    df["task2_hard_labels"] = task_encoder.inverse_transform(df["task2_hard_labels"])
    return df

In [23]:
import optuna
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset

In [24]:
#load given data
import pandas as pd
og_train1 = train_df
og_dev1 = dev_df

In [25]:
all_task_hard_labels = pd.concat([og_train1["task2_hard_labels"],og_dev1["task2_hard_labels"]])

In [26]:
train1_df = task_hard_encode(og_train1)

In [27]:
train1_df.head()

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,ethnicities_annotators,study_levels_annotators,countries_annotators,...,labels_task3,split,lang_label,annotator_number_list,gender_label_list,age_annotators_list,task2_labels_list,task2_labels_string,task2_soft_labels,task2_hard_labels
0,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...",6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[White or Caucasian, Hispano or Latino, White ...","[Bachelor’s degree, Bachelor’s degree, High sc...","[Italy, Mexico, United States, Spain, Spain, C...",...,"[[OBJECTIFICATION], [OBJECTIFICATION, SEXUAL-V...",TRAIN_ES,0,"[1, 2, 3, 4, 5, 6]","[0, 0, 0, 1, 1, 1]","[0, 1, 2, 2, 1, 0]","[2, 3, 0, 2, 3, 2]",12302321,"[0.0, 0.3333333333333333, 0.16666666666666666,...",3
1,100002,es,@ultimonomada_ Si comicsgate se parece en algo...,6,"[Annotator_7, Annotator_8, Annotator_9, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[Black or African American, Hispano or Latino,...","[High school degree or equivalent, Bachelor’s ...","[United Kingdom, Mexico, United States, Portug...",...,"[[-], [-], [-], [-], [OBJECTIFICATION], [-]]",TRAIN_ES,0,"[7, 8, 9, 10, 11, 12]","[0, 0, 0, 1, 1, 1]","[0, 1, 2, 2, 1, 0]","[0, 0, 0, 0, 1, 0]",10000101,"[0.16666666666666666, 0.0, 0.8333333333333334,...",0
2,100003,es,"@Steven2897 Lee sobre Gamergate, y como eso ha...",6,"[Annotator_7, Annotator_8, Annotator_9, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[Black or African American, Hispano or Latino,...","[High school degree or equivalent, Bachelor’s ...","[United Kingdom, Mexico, United States, Portug...",...,"[[-], [-], [-], [-], [-], [-]]",TRAIN_ES,0,"[7, 8, 9, 10, 11, 12]","[0, 0, 0, 1, 1, 1]","[0, 1, 2, 2, 1, 0]","[0, 0, 0, 0, 0, 0]",10000001,"[0.0, 0.0, 1.0, 0.0]",0
3,100004,es,@Lunariita7 Un retraso social bastante lamenta...,6,"[Annotator_13, Annotator_14, Annotator_15, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[Hispano or Latino, Hispano or Latino, White o...","[High school degree or equivalent, Bachelor’s ...","[Mexico, Chile, Spain, Spain, Portugal, Spain]",...,"[[-], [-], [IDEOLOGICAL-INEQUALITY], [-], [IDE...",TRAIN_ES,0,"[13, 14, 15, 16, 17, 18]","[0, 0, 0, 1, 1, 1]","[0, 1, 2, 2, 1, 0]","[0, 0, 1, 0, 2, 2]",10010221,"[0.16666666666666666, 0.0, 0.5, 0.333333333333...",0
4,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,6,"[Annotator_19, Annotator_20, Annotator_21, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[Hispano or Latino, Hispano or Latino, White o...","[Bachelor’s degree, Bachelor’s degree, Master’...","[Mexico, Afghanistan, United States, Italy, Po...",...,"[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [-...",TRAIN_ES,0,"[19, 20, 21, 22, 23, 24]","[0, 0, 0, 1, 1, 1]","[0, 1, 2, 2, 1, 0]","[2, 0, 3, 0, 3, 1]",12030311,"[0.16666666666666666, 0.3333333333333333, 0.33...",4


In [28]:
train1_df = train1_df[["tweet","task2_hard_labels"]].dropna()

In [29]:
train1_df = train1_df[train1_df['task2_hard_labels'] != 4]

In [30]:
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

In [31]:
test1_df = task_hard_encode(og_dev1)
test1_df = test1_df[["tweet","task2_hard_labels"]].dropna()
test1_df = test1_df[test1_df['task2_hard_labels'] != 4] #removing nans
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

In [32]:
print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)

train1 (6064, 2)
<bound method NDFrame.head of                                                   tweet  task2_hard_labels
0     @USER Ignora al otro, es un capullo.El problem...                  3
1     @USER Si comicsgate se parece en algo a gamerg...                  0
2     @USER Lee sobre Gamergate, y como eso ha cambi...                  0
3     @USER Un retraso social bastante lamentable, g...                  0
5     @USER Aaah sí. Andrew Dobson. El que se dedicó...                  0
...                                                 ...                ...
6913  @USER Ma'am if I say that you look like a whor...                  1
6915  idk why y’all bitches think having half your a...                  1
6916  This has been a part of an experiment with @US...                  2
6917  "Take me already" "Not yet. You gotta be ready...                  1
6918    @USER why do you look like a whore? /lh HTTPURL                  1

[6064 rows x 2 columns]>
test1 (910, 2)
<bound metho

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, test1_df], ignore_index=True)

In [34]:
# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

In [35]:
id2label = { "0":"-" ,"1":"DIRECT", "2":"REPORTED","3":"JUDGEMENTAL"}
label2id = {"-": "0", "DIRECT":"1", "REPORTED":"2","JUDGEMENTAL":"3"}


In [36]:
# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, temp_df = train_test_split(combined_df_shuffled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [37]:
# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [38]:
# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4,ignore_mismatched_sizes=True, id2label=id2label, label2id=label2id)

tokenizer_config.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.

In [39]:
# Set parameters
LR_MIN = 1e-6
LR_CEIL = 0.001
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 3
MAX_EPOCHS = 10
PER_DEVICE_EVAL_BATCH = 4
PER_DEVICE_TRAIN_BATCH = 4
NUM_TRIALS = 3
SAVE_DIR = '/kaggle/working/optuna_models/'
NAME_OF_MODEL = 'sexist_tweet_predictor_task2_hard'
MAX_LENGTH = 128

In [40]:
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["task2_hard_labels"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

In [41]:
# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)

In [42]:
# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "label": test_dataset["label"]})

In [43]:
def objective(trial):
    training_args = TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', LR_MIN, LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', MIN_EPOCHS, MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        evaluation_strategy="epoch",  # Evaluate at the end of each epoch
        save_strategy="epoch",  # Save at the end of each epoch
        load_best_model_at_end=True,  # Load the best model found during training at the end
        save_total_limit=1,  # Only save the best checkpoint
        disable_tqdm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    result = trainer.train()
    return result.training_loss

In [44]:
study = optuna.create_study(study_name='hp-search-task2-hard', direction='minimize')
study.optimize(objective, n_trials=NUM_TRIALS)

[I 2024-05-06 14:45:55,809] A new study created in memory with name: hp-search-task2-hard
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.015500,0.965787
2,0.992400,0.973308
3,1.010100,0.967825


[I 2024-05-06 15:08:08,295] Trial 0 finished with value: 1.0114471690771543 and parameters: {'learning_rate': 5.643941095852829e-05, 'weight_decay': 0.009619600174266142, 'num_train_epochs': 3}. Best is trial 0 with value: 1.0114471690771543.


Epoch,Training Loss,Validation Loss
1,0.962200,0.962862
2,0.993000,0.982113
3,1.013300,0.984161
4,0.988400,0.968597
5,0.986100,0.963264
6,0.978800,0.972307
7,0.981500,0.967054


[I 2024-05-06 15:58:27,140] Trial 1 finished with value: 0.9947870257446476 and parameters: {'learning_rate': 3.063615218309414e-05, 'weight_decay': 6.927086765322737e-05, 'num_train_epochs': 7}. Best is trial 1 with value: 0.9947870257446476.


Epoch,Training Loss,Validation Loss
1,0.914200,0.961508
2,0.978600,0.963549
3,1.011400,0.960370


[I 2024-05-06 16:19:47,944] Trial 2 finished with value: 0.9721983331933244 and parameters: {'learning_rate': 5.278312990544521e-06, 'weight_decay': 0.0006364809904203995, 'num_train_epochs': 3}. Best is trial 2 with value: 0.9721983331933244.


In [46]:
print_custom('Finding study best parameters')    
best_lr = float(study.best_params['learning_rate'])
best_weight_decay = float(study.best_params['weight_decay'])
best_epoch = int(study.best_params['num_train_epochs'])
print_custom('Extract best study params')

print(f'The best learning rate is: {best_lr}')
print(f'The best weight decay is: {best_weight_decay}')
print(f'The best epoch is : {best_epoch}')

print_custom('Create dictionary of the best hyperparameters')
best_hp_dict = {
    'best_learning_rate' : best_lr,
    'best_weight_decay': best_weight_decay,
    'best_epoch': best_epoch
}



Finding study best parameters
----------------------------------------------------------------------------------------------------


Extract best study params
----------------------------------------------------------------------------------------------------
The best learning rate is: 5.278312990544521e-06
The best weight decay is: 0.0006364809904203995
The best epoch is : 3


Create dictionary of the best hyperparameters
----------------------------------------------------------------------------------------------------
